In [ ]:
import os
import flopy
import pyemu
import pandas as pd
from apexmf_pkgs import apexmf_pst_utils, apexmf_pst_par

In [ ]:
wd = "D:/Workshops/2022_Webinar_apexmf_opt_BLM/apexmf_1st_cal/APEX-MODFLOW"
mfwd = "D:/Workshops/2022_Webinar_apexmf_opt_BLM/apexmf_1st_cal/APEX-MODFLOW/MODFLOW"
os.chdir(wd)
mname = "modflow.mfn"

### Model Setup

- Simulation Period 
    * Jan. 01, 2002 ~ Dec. 31, 2002 with no warm-up period.
- Measurement Duration
    * Streamflow - Jan. 01, 2002 ~ Dec. 31, 2002
    * Watertable - vary in 2002
    <br/><br/>
- Calibration / Validation
    * 1/1/2002 - 12/31/2002 for Monthly average stream discharge (1 year)
    * 1/1/2002 - 12/31/2002 for daily depth to water (1 year)
    <br/><br/>

    - APEX parameters:
    p12, p16, p17, p91, p92, p95
    - MODFLOW parameters:
    hk01, hk02, sy01, sy02

### 1. Create PEST input files (template / instruction)

#### 1.1. Create template files
We are going to use the *.pval and mf_river.par files for MODFLOW parameters and model.in file for SWAT parameters.

In [ ]:
# pval file
pval_file = os.path.join(mfwd, 'mf_1000.pval')
# parmfile
parm_file = 'PARM1501.DAT'

In [ ]:
gw_par = pyemu.utils.gw_utils.modflow_pval_to_template_file(pval_file, tpl_file=None)
print(gw_par)

In [ ]:
# Create parm template file
sw_par = apexmf_pst_utils.parm_to_tpl_file()
sw_par

## 1.2. Build instruction files (streamflow / watertable / baseflow)
### 1.2.1. Streamflow (output.rch)

In [ ]:
# file path
rch_file = 'SITE75.RCH'
# reach numbers that are used for calibration
subs = [12, 57, 75]


In [ ]:
# extract month_streamflow
apexmf_pst_utils.extract_month_str(rch_file, subs, '1/1/2002', '1/1/2002', '12/31/2002')

In [ ]:
# apexmf_pst_utils.extract_month_sed(rch_file, subs, '1/1/1980', '1/1/1992', '12/31/1999')

### 1.2.3. Create instruction files for each str_sim file using the 'streamflow.obd' file

In [ ]:
# because we have 3 streamgages let's loop for them
# read streamobd and get column names
stf_obd = pd.read_csv('stf_mon.obd',sep='\t', index_col=0,parse_dates=True,na_values=[-999, ''])
obds = stf_obd.columns.tolist()
print(obds)
sim_files = ['stf_{:03d}.txt'.format(x) for x in subs]
print(sim_files)

In [ ]:
apexmf_pst_utils.stf_obd_to_ins?

In [ ]:
# create instruction files for each sim file
for i in range(len(sim_files)):
    apexmf_pst_utils.stf_obd_to_ins(sim_files[i], obds[i], '1/1/2002', '12/31/2002', time_step='month')

In [ ]:
# We do have watertable data now
grid_ids = [5895, 6273]
apexmf_pst_utils.extract_depth_to_water(grid_ids, '1/1/2002', '12/31/2002')



In [ ]:
apexmf_pst_utils.mf_obd_to_ins?

In [ ]:
apexmf_pst_utils.mf_obd_to_ins('dtw_5895.txt', 'gw_124', '1/1/2002', '12/31/2002')
apexmf_pst_utils.mf_obd_to_ins('dtw_6273.txt', 'gw_098', '1/1/2002', '12/31/2002')

## Create a dummy pst file 

In [ ]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)
# pyemu.helpers.pst_from_io_files(io_files[0], io_files[1], io_files[2], io_files[3], 'ani_dummy.pst')

# print(os.chdir(".."))
io_files

In [ ]:
io_files

In [ ]:
pyemu.helpers.pst_from_io_files?

The ``parse_dir_for_io_files()`` helper is looking for files with the ".tpl" and ".ins" extension.  This assumes that the corresponding model input and model output files are the same name, minus the ".tpl" and ".ins" extension, respectively.  These file lists are then passed to another helper, which builds a basic control file for you (``Pst.from_io_files()``).  Let's look at this generic ``Pst`` instance:

In [ ]:
par = pst.parameter_data
par

#### 2.1. Change parameter group name

In [ ]:
for i in range(len(par)):
    if (par.iloc[i, 0][:2]) == 'sy':
        par.iloc[i, 6] = 'sy'
    elif par.iloc[i, 0][:2] == 'hk':
        par.iloc[i, 6] = 'hk'
    elif par.iloc[i, 0][:1] == 'p':
        par.iloc[i, 6] = 'apex'
print(par)

# 2.2. Set par ranges and initial values for parameters

### 2.2.3. MODFLOW

### Let's start values from 1st round of calibration result

In [ ]:
# for MODFLOW parameters
count = 0
for i in range(len(par)):
    if (par.iloc[i, 6] == 'hk') and (par.iloc[i, 0] == 'hk01'):  
        par.iloc[i, 3] = 0.01    
        par.iloc[i, 4] = 0.01 / 100
        par.iloc[i, 5] = 0.01 * 100
    elif (par.iloc[i, 6] == 'hk') and (par.iloc[i, 0] == 'hk02'):  
        par.iloc[i, 3] = 5   
        par.iloc[i, 4] = 5 / 100
        par.iloc[i, 5] = 5 * 100
    elif (par.iloc[i, 6] == 'sy') and (par.iloc[i, 0] == 'sy01'):  
        par.iloc[i, 3] = 0.1      
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 5.000000e-01
    elif (par.iloc[i, 6] == 'sy') and (par.iloc[i, 0] == 'sy02'): 
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 5.000000e-01
    else:
        count += 1
print(count)

In [ ]:
par

# APEX

In [ ]:
pst.parameter_data = apexmf_pst_utils.export_pardb_pest(par)


In [ ]:
par = pst.parameter_data
par

## Observation

In [ ]:
# set observation group
obd = pst.observation_data
obd

In [ ]:
# Change obd group name
for i in range(len(obd)):
    if obd.iloc[i, 0][:2] == 'gw':
        obd.iloc[i, 3] = obd.iloc[i, 0][:6]
    else:
        obd.iloc[i, 3] = obd.iloc[i, 0][:-7]
obd

## 2.3. Import measured data

In [ ]:
gwt_obd = pd.read_csv('MODFLOW/dtw_day.obd',
                       sep='\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
gwt_obd = gwt_obd['1/1/2002': '12/31/2002']
gwt_obd = gwt_obd[['gw_098', 'gw_124']]
gwt_obd

In [ ]:
stf_obd = pd.read_csv('stf_mon.obd',
                       sep='\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
stf_obd = stf_obd['1/1/2002': '12/31/2002']
stf_obd




In [ ]:
# Get sub list based on obd order
sub_order = []
for i in obd.obgnme.tolist():
    if i not in sub_order:
        sub_order.append(i)
sub_order

In [ ]:
# get total list from each sub obd, delete na vals
tot_obd = []
for i in sub_order[:2]:
    tot_obd += gwt_obd[i].dropna().tolist()
for j in sub_order[2:]:
    tot_obd += stf_obd[j].dropna().tolist()    
len(tot_obd)
# tot_obd

In [ ]:
obd.loc[:, 'obsval'] = tot_obd
obd

### 4. Export control file

In [ ]:
pst.control_data.noptmax=0

In [ ]:
pst.model_command = 'python forward_run.py'

In [ ]:
pst.write('ani_pest.pst')